# test-model

Test model accuracy against a couple images it never saw during training.

In [1]:
from PIL import Image
from typing import List

import pandas as pd

import torch
import torch.nn as nn
import torchvision.models as models

from torchvision.transforms import v2

In [2]:
IMG_SIZE = 256

BAND_COLORS = [
    'silver', 'white', 'blue', 'grey', 'violet', 'green', 
    'yellow', 'orange', 'red', 'gold', 'black', 'brown'
]
MAX_BANDS = 4
CLASSES = MAX_BANDS * len(BAND_COLORS)

In [3]:
model = models.efficientnet_v2_m(weights=models.EfficientNet_V2_M_Weights.DEFAULT)
model.classifier[1] = nn.Linear(in_features=model.classifier[1].in_features, out_features=CLASSES)

In [4]:
model.load_state_dict(torch.load('../what-the-ohm-4B.pth'))

<All keys matched successfully>

In [5]:
# generate list of classes
def gen_classes() -> List[str]:
    classes = []
    for i in range(1, MAX_BANDS + 1):
        for color in BAND_COLORS:
            classes.append(f'band_{i}_{color}')
    return classes

# use model to predict bands in image
def predict(model: nn.Module, img_path: str) -> torch.Tensor:
    preprocess = v2.Compose([
        v2.Resize(IMG_SIZE),
        v2.ToImage(),
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ])

    img = Image.open(img_path)
    img_tensor = preprocess(img)
    batch = img_tensor.unsqueeze(0) # add dummy batch dimension

    model.eval()

    with torch.no_grad():
        logits = model(batch).squeeze(0) # remove dummy batch dimension
        class_probs = torch.sigmoid(logits)
    return class_probs

# convert band probabilities to classes
def probs_to_classes(probs: torch.Tensor, threshold: float = 0.5) -> List[str]:
    classes = gen_classes()

    for i,c in enumerate(classes):
        if i > 0 and i % len(BAND_COLORS) == 0:
            print()
        p = probs[i] * 100.0
        print(f'{c} -> {p:.2f}%')

    pred_indices = (probs > threshold).nonzero()[:, 0]
    return [classes[i] for i in pred_indices]

# test model and print results
def test_model(model: nn.Module, img_path: str, actual_classes: List[str]):
    probs = predict(model, img_path)
    pred_classes = probs_to_classes(probs)

    print('\npredicted classes:', pred_classes)
    print('actual classes:   ', actual_classes)


## Test 1

4 band, 1M ohm, 5% tolerance

<img src="test-data/4B-1M-T5.jpg" alt="test-data/4B-1M-T5.jpg" width="256"/>

In [6]:
test_model(model, 'test-data/4B-1M-T5.jpg', actual_classes=['band_1_brown', 'band_2_black', 'band_3_green', 'band_4_gold'])

band_1_silver -> 0.19%
band_1_white -> 0.17%
band_1_blue -> 0.35%
band_1_grey -> 0.63%
band_1_violet -> 0.15%
band_1_green -> 0.32%
band_1_yellow -> 0.94%
band_1_orange -> 0.66%
band_1_red -> 1.77%
band_1_gold -> 0.18%
band_1_black -> 0.49%
band_1_brown -> 99.80%

band_2_silver -> 0.17%
band_2_white -> 0.47%
band_2_blue -> 0.33%
band_2_grey -> 0.82%
band_2_violet -> 0.71%
band_2_green -> 35.60%
band_2_yellow -> 0.12%
band_2_orange -> 0.87%
band_2_red -> 5.00%
band_2_gold -> 0.16%
band_2_black -> 96.89%
band_2_brown -> 0.49%

band_3_silver -> 0.17%
band_3_white -> 0.30%
band_3_blue -> 0.39%
band_3_grey -> 0.22%
band_3_violet -> 0.23%
band_3_green -> 99.72%
band_3_yellow -> 0.89%
band_3_orange -> 7.86%
band_3_red -> 1.90%
band_3_gold -> 7.93%
band_3_black -> 1.78%
band_3_brown -> 5.37%

band_4_silver -> 0.16%
band_4_white -> 0.25%
band_4_blue -> 0.13%
band_4_grey -> 0.33%
band_4_violet -> 0.17%
band_4_green -> 0.18%
band_4_yellow -> 0.12%
band_4_orange -> 0.20%
band_4_red -> 0.15%
band_4

## Test 2

4 band, 4.7K ohm, 5% tolerance

<img src="test-data/4B-4K7-T5.jpg" alt="test-data/4B-4K7-T5.jpg" width="256"/>

In [7]:
test_model(model, 'test-data/4B-4K7-T5.jpg', actual_classes=['band_1_yellow', 'band_2_violet', 'band_3_red', 'band_4_gold'])

band_1_silver -> 0.29%
band_1_white -> 0.61%
band_1_blue -> 4.67%
band_1_grey -> 1.37%
band_1_violet -> 0.21%
band_1_green -> 0.27%
band_1_yellow -> 99.85%
band_1_orange -> 0.86%
band_1_red -> 2.05%
band_1_gold -> 0.25%
band_1_black -> 0.70%
band_1_brown -> 1.28%

band_2_silver -> 0.36%
band_2_white -> 2.28%
band_2_blue -> 0.36%
band_2_grey -> 5.20%
band_2_violet -> 99.79%
band_2_green -> 0.44%
band_2_yellow -> 0.29%
band_2_orange -> 3.98%
band_2_red -> 0.63%
band_2_gold -> 0.32%
band_2_black -> 0.97%
band_2_brown -> 0.68%

band_3_silver -> 0.24%
band_3_white -> 0.25%
band_3_blue -> 0.33%
band_3_grey -> 0.27%
band_3_violet -> 0.40%
band_3_green -> 0.63%
band_3_yellow -> 2.11%
band_3_orange -> 15.24%
band_3_red -> 6.13%
band_3_gold -> 21.44%
band_3_black -> 67.63%
band_3_brown -> 13.81%

band_4_silver -> 0.13%
band_4_white -> 0.34%
band_4_blue -> 0.21%
band_4_grey -> 0.39%
band_4_violet -> 0.19%
band_4_green -> 0.37%
band_4_yellow -> 0.25%
band_4_orange -> 0.31%
band_4_red -> 0.20%
band

## Test 3

4 band, 47K ohm, 5% tolerance

<img src="test-data/4B-47K-T5.jpg" alt="test-data/4B-47K-T5.jpg" width="256"/>

In [8]:
test_model(model, 'test-data/4B-47K-T5.jpg', actual_classes=['band_1_yellow', 'band_2_violet', 'band_3_orange', 'band_4_gold'])

band_1_silver -> 0.42%
band_1_white -> 0.30%
band_1_blue -> 0.83%
band_1_grey -> 0.28%
band_1_violet -> 0.17%
band_1_green -> 0.70%
band_1_yellow -> 99.18%
band_1_orange -> 4.89%
band_1_red -> 3.36%
band_1_gold -> 0.24%
band_1_black -> 0.33%
band_1_brown -> 0.80%

band_2_silver -> 0.38%
band_2_white -> 9.57%
band_2_blue -> 0.96%
band_2_grey -> 0.63%
band_2_violet -> 99.48%
band_2_green -> 1.67%
band_2_yellow -> 0.20%
band_2_orange -> 3.84%
band_2_red -> 0.62%
band_2_gold -> 0.30%
band_2_black -> 0.39%
band_2_brown -> 0.84%

band_3_silver -> 0.22%
band_3_white -> 0.17%
band_3_blue -> 0.19%
band_3_grey -> 0.29%
band_3_violet -> 0.42%
band_3_green -> 0.49%
band_3_yellow -> 7.85%
band_3_orange -> 98.42%
band_3_red -> 0.65%
band_3_gold -> 14.63%
band_3_black -> 3.74%
band_3_brown -> 1.45%

band_4_silver -> 0.13%
band_4_white -> 0.19%
band_4_blue -> 0.20%
band_4_grey -> 0.42%
band_4_violet -> 0.09%
band_4_green -> 0.28%
band_4_yellow -> 0.26%
band_4_orange -> 0.30%
band_4_red -> 0.11%
band_4

## Test 4

4 band, 220 ohm, 5% tolerance

<img src="test-data/4B-220R-T5.jpg" alt="test-data/4B-220R-T5.jpg" width="256"/>

In [9]:
test_model(model, 'test-data/4B-220R-T5.jpg', actual_classes=['band_1_red', 'band_2_red', 'band_3_brown', 'band_4_gold'])

band_1_silver -> 1.32%
band_1_white -> 0.57%
band_1_blue -> 1.72%
band_1_grey -> 10.62%
band_1_violet -> 0.72%
band_1_green -> 3.21%
band_1_yellow -> 0.61%
band_1_orange -> 14.82%
band_1_red -> 99.54%
band_1_gold -> 0.68%
band_1_black -> 0.64%
band_1_brown -> 1.13%

band_2_silver -> 0.52%
band_2_white -> 1.84%
band_2_blue -> 2.88%
band_2_grey -> 1.04%
band_2_violet -> 2.61%
band_2_green -> 8.51%
band_2_yellow -> 0.63%
band_2_orange -> 23.59%
band_2_red -> 99.49%
band_2_gold -> 1.00%
band_2_black -> 0.41%
band_2_brown -> 2.17%

band_3_silver -> 0.75%
band_3_white -> 0.50%
band_3_blue -> 0.53%
band_3_grey -> 0.84%
band_3_violet -> 0.83%
band_3_green -> 35.09%
band_3_yellow -> 2.00%
band_3_orange -> 19.44%
band_3_red -> 5.44%
band_3_gold -> 28.79%
band_3_black -> 8.28%
band_3_brown -> 78.60%

band_4_silver -> 0.66%
band_4_white -> 0.55%
band_4_blue -> 0.48%
band_4_grey -> 0.56%
band_4_violet -> 0.72%
band_4_green -> 1.06%
band_4_yellow -> 0.73%
band_4_orange -> 0.89%
band_4_red -> 1.17%
b

## Test 5

4 band, 22K ohm, 5% tolerance

<img src="test-data/4B-22K-T5.jpg" alt="test-data/4B-22K-T5.jpg" width="256"/>

In [10]:
test_model(model, 'test-data/4B-22K-T5.jpg', actual_classes=['band_1_red', 'band_2_red', 'band_3_orange', 'band_4_gold'])

band_1_silver -> 0.35%
band_1_white -> 0.16%
band_1_blue -> 2.24%
band_1_grey -> 1.16%
band_1_violet -> 0.26%
band_1_green -> 0.96%
band_1_yellow -> 1.57%
band_1_orange -> 15.41%
band_1_red -> 99.10%
band_1_gold -> 0.25%
band_1_black -> 0.26%
band_1_brown -> 0.97%

band_2_silver -> 0.19%
band_2_white -> 1.48%
band_2_blue -> 1.03%
band_2_grey -> 1.58%
band_2_violet -> 2.89%
band_2_green -> 0.66%
band_2_yellow -> 0.24%
band_2_orange -> 25.68%
band_2_red -> 97.69%
band_2_gold -> 0.27%
band_2_black -> 0.23%
band_2_brown -> 0.43%

band_3_silver -> 0.41%
band_3_white -> 0.17%
band_3_blue -> 0.15%
band_3_grey -> 0.20%
band_3_violet -> 0.45%
band_3_green -> 2.58%
band_3_yellow -> 1.57%
band_3_orange -> 99.77%
band_3_red -> 1.13%
band_3_gold -> 1.26%
band_3_black -> 3.47%
band_3_brown -> 2.93%

band_4_silver -> 0.19%
band_4_white -> 0.13%
band_4_blue -> 0.27%
band_4_grey -> 0.25%
band_4_violet -> 0.15%
band_4_green -> 0.25%
band_4_yellow -> 0.22%
band_4_orange -> 0.24%
band_4_red -> 0.33%
band_